<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/3c_minhash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de documentos con MinHash
En esta libreta veremos cómo hacer búsqueda eficiente de documentos considerando la similitud de Jaccard.

La similitud de Jaccard entre un par de conjuntos $(\mathcal{C}^{(1)}, \mathcal{C}^{(2)})$ está dada por

$$
J(\mathcal{C}^{(1)}, \mathcal{C}^{(2)}) = \frac{\mid \mathcal{C}^{(1)} \cap \mathcal{C}^{(2)} \mid}{\mid \mathcal{C}^{(1)}\cup \mathcal{C}^{(2)} \mid} \in [0,1]
$$

In [1]:
from collections import Counter
from math import floor, log
import codecs 
import re 

import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

VOCMAX = 5000
n_muestras_sint = 10000
n_tablas_ng = 50

# Para reproducibilidad
np.random.seed(2021)

## Conjuntos de datos
Preparamos dos conjuntos de datos para probar los algoritmos de búsqueda.

### Datos sintéticos
Este conjunto de datos está compuesto por 5 listas de elementos y un conjunto universo de 10 elementos.


In [2]:
sint_conj = [[0, 1, 4, 6, 8], [2, 3, 4, 7, 8], [1, 4, 6, 7], [0, 5, 6, 8], [0, 1, 3, 4, 7]]
univ = {e for l in sint_conj for e in l}

Calculamos la similitud de Jaccard de todos los pares.

In [3]:
sims_jacc = np.identity(len(sint_conj))
for i in range(0, len(sint_conj) - 1):
  ci = set(sint_conj[i])
  for j in range(i + 1, len(sint_conj)):
    cj = set(sint_conj[j])
    sims_jacc[i,j] = float(len(ci.intersection(cj)) / len(ci.union(cj)))
    sims_jacc[j,i] = sims_jacc[i,j]

print(sims_jacc)

[[1.         0.25       0.5        0.5        0.42857143]
 [0.25       1.         0.28571429 0.125      0.42857143]
 [0.5        0.28571429 1.         0.14285714 0.5       ]
 [0.5        0.125      0.14285714 1.         0.125     ]
 [0.42857143 0.42857143 0.5        0.125      1.        ]]


Definimos una función que calcula la propiedad de colisión de todos los pares en este conjunto de datos

In [4]:
def p_colision(db, tablas, n_tablas, n_ej):
  for i in range(n_tablas):
    for j,l in enumerate(db):
      tablas[i].insertar(l, j)

  colisiones = np.zeros((n_ej, n_ej))
  for i in range(n_tablas):
    for j,cj in enumerate(db):
      for e in tablas[i].buscar(cj):
        colisiones[j, e] += 1

  return colisiones / n_tablas

### 20 Newsgroups
Vamos a usar el conjunto de documentos de _20 Newsgropus_, el cual descargamos usando scikit-learn. 

In [5]:
db = fetch_20newsgroups(remove=('headers','footers','quotes'))

Importamos la biblioteca NLTK y definimos nuestro analizador léxico y lematizador

In [6]:
import nltk
nltk.download(['punkt','averaged_perceptron_tagger','wordnet'])

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.corpus.reader.wordnet import NOUN, VERB, ADV, ADJ

morphy_tag = {
    'JJ' : ADJ,
    'JJR' : ADJ,
    'JJS' : ADJ,
    'VB' : VERB,
    'VBD' : VERB,
    'VBG' : VERB,
    'VBN' : VERB,
    'VBP' : VERB,
    'VBZ' : VERB,
    'RB' : ADV,
    'RBR' : ADV,
    'RBS' : ADV
}

def doc_a_tokens(doc):
  tagged = pos_tag(word_tokenize(doc.lower()))
  lemmatizer = WordNetLemmatizer()
  tokens = []
  for p,t in tagged:
    tokens.append(lemmatizer.lemmatize(p, pos=morphy_tag.get(t, NOUN)))

  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Convertimos el conjunto preprocesado a una lista de cadenas, una por documento

In [7]:
corpus = []
for d in db.data:
  d = d.replace('\n',' ').replace('\r',' ').replace('\t',' ')
  d = ' '.join([''.join([c.lower() for c in p if c.isalnum()]) for p in d.split()])
  tokens = doc_a_tokens(d)
  corpus.append(' '.join(tokens))

Dividimos nuestro conjunto en 2 subconjuntos: los documentos de la base que se buscarán y los documentos de consulta

In [8]:
perm = np.random.permutation(len(corpus)).astype(int)
n_ej_base = int(floor(len(corpus) * 0.95))

base = [corpus[i] for i in perm[:n_ej_base]]
consultas = [corpus[i] for i in perm[n_ej_base:]]

Obtenemos y cargamos la lista de _stopwords_ para inglés (archivo con una palabra por línea)

In [9]:
!wget -qO- -O stopwords_english.txt \
         https://raw.githubusercontent.com/pan-webis-de/authorid/master/data/stopwords_english.txt

stopwords = []
for line in codecs.open('stopwords_english.txt', encoding = "utf-8"):
  stopwords.append(line.rstrip())

Procesamos cada palabra del corpus completo para generar y ordenar el vocabulario

In [10]:
# Divide la cadena en palabras
term_re = re.compile("\w+", re.UNICODE)

# Contamos las ocurrencias de cada palabra
corpus_freq = Counter()
doc_freq = Counter()
for d in base:
  # Eliminamos números de la cadena (documento) a procesar 
  d = re.sub(r'\d+', '', d)

  # Dividimos la cadena en una lista de palabras
  terms = [t for t in term_re.findall(d) if t not in stopwords and len(t) > 2]
  
  # Aumentamos el contador de cada instancia palabra en el documento
  for t in terms:
    corpus_freq[t] += 1
  
  # Aumentamos el contador de cada palabra distinta en el documento
  for t in set(terms):
    doc_freq[t] += 1

# Generamos un diccionario con las VOCMAX palabras más frecuentes
vocabulary = {entry[0]:(i, entry[1], doc_freq[entry[0]], log(len(corpus) / doc_freq[entry[0]])) \
              for i, entry in enumerate(corpus_freq.most_common()) \
              if i < VOCMAX}

Creamos un diccionario para mapear índices a palabras

In [11]:
id_a_palabra = {v[0]: k for k,v in vocabulary.items()}

Generamos las bolsas de palabras de los documentos preprocesados

In [12]:
def cadenas_a_bolsas(cadenas, voc, descartar, tre):
  bolsas = []
  for c in cadenas:
    c = re.sub(r'\d+', '', c)
    ids = Counter([voc[t][0] for t in tre.findall(c) \
                   if t in voc and t not in descartar])
    bolsas.append([i for i in sorted(ids.items())])

  return bolsas

bolsas_base = cadenas_a_bolsas(base, vocabulary, stopwords, term_re)
bolsas_consultas = cadenas_a_bolsas(consultas, vocabulary, stopwords, term_re)

## MinHash binario
Min-Hashing es un algoritmo para búsqueda de conjuntos similares bajo la similitud de Jaccard, la cual consiste en:
* Generar permutación aleatoria del conjunto universo $\mathbb{U}$
* Asignar a cada conjunto su 1er elemento bajo la permutación, esto es,
$$
h(\mathcal{C}^{(i)}) = min(\pi(\mathcal{C}^{(i)}))
$$

La probabilidad de que dos conjuntos tengan valor MinHash idéndico es igual a su similitud de Jaccard:
$$
P[h(\mathcal{C}^{(i)}) = h(\mathcal{C}^{(j)})] = \frac{\mid \mathcal{C}^{(i)} \cap \mathcal{C}^{(j)} \mid}{\mid \mathcal{C}^{(i)}\cup \mathcal{C}^{(j)} \mid} \in [0,1]
$$

Para buscar conjuntos similares, los valores MinHash se agrupan en $l$ tuplas de
$r$ funciones distintas de la siguiente forma:    

\begin{align*}
 g_1(\mathcal{C}^{(i)}) & = (h_1(\mathcal{C}^{(i)}), h_2(\mathcal{C}^{(i)}), \ldots , h_r(\mathcal{C}^{(i)}))\\
g_2(\mathcal{C}^{(i)}) & = (h_{r+1}(\mathcal{C}^{(i)}), h_{r+2}(\mathcal{C}^{(i)}), \ldots , h_{2\cdot r}(\mathcal{C}^{(i)}))\\
      \cdots\\
g_l(\mathcal{C}^{(i)}) & = (h_{(l-1)\cdot r+1}(\mathcal{C}^{(i)}), h_{(l-1)\cdot r2}(\mathcal{C}^{(i)}), \ldots , h_{l\cdot r}(\mathcal{C}^{(i)}))
\end{align*}
    
Conjuntos con una tupla idéntica se almacenan en la misma cubeta en la tabla asociada a la tupla.

In [13]:
class MinHashTable:
  def __init__(self, n_cubetas, t_tupla, dim):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.dim = dim
    self.t_tupla = t_tupla
    
    self.perm = np.random.uniform(0, 1, size=(self.t_tupla, self.dim))
    self.rind = np.random.randint(0, np.iinfo(np.int32).max, size=(self.t_tupla, self.dim))

    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def minhash(self, x):
    xp = self.perm[:, x]
    xi = self.rind[:, x]
    amin = xp.argmin(axis = 1)
    emin = xi[:, amin]

    return np.sum(self.a * emin, dtype=np.ulonglong), np.sum(self.b * emin, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    mh, v2 = self.minhash(x)
  
    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(mh)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == mh:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

  def eliminar(self, x, ident):
    mh, v2 = self.minhash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        break
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1].remove(ident)

    return -1

### Verificación con conjunto de datos sintéticos
Primero verificamos la probabilidad de colisión de dos conjuntos en nuestra implementación.

In [14]:
tablas_sint_bin = [MinHashTable(2**4, 1, len(univ)) for _ in range(n_muestras_sint)]
print(p_colision(sint_conj, tablas_sint_bin, n_muestras_sint, len(sint_conj)))

[[1.     0.2471 0.5046 0.5002 0.4253]
 [0.2471 1.     0.2827 0.1235 0.4242]
 [0.5046 0.2827 1.     0.1477 0.5002]
 [0.5002 0.1235 0.1477 1.     0.1224]
 [0.4253 0.4242 0.5002 0.1224 1.    ]]


### Búsqueda de documentos similares con _20 newsgroups_
Probamos la implementación de Min-Hashing para la búsqueda de documentos similares en _20 newsgroups_. Para realizar la búsqueda de documentos similares:

1. Insertamos las listas a nuestras tablas
2. Recuperamos los documentos similares a nuestros documentos de consulta usando las tablas MinHash.
3. Calculamos la similitud Jaccard de los documentos recuperados con los de consulta 
4. Ordenamos por similitud.

In [15]:
def similitud_jaccard(x, y):
  xa = np.zeros(VOCMAX)
  xa[x] = 1
  ya = np.zeros(VOCMAX)
  xa[x] = 1
  
  inter = np.count_nonzero(x * y)
  return inter / (np.count_nonzero(x) + np.count_nonzero(y) - inter)

def fuerza_bruta(ds, qs, fs):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fs(qs, x)
  return np.sort(medidas)[::-1], np.argsort(medidas)[::-1]

def busca_pares_documentos(ll_base, ll_consulta, tablas, fs):
  for j,l in enumerate(ll_base):
    for i in range(len(tablas)):
      tablas[i].insertar(l, j)

  docs = []
  for j,l in enumerate(ll_consultas):
    dc = []
    for i in range(len(tablas)):
      dc.extend(tablas[i].buscar(l))
    docs.append(set(dc))

  sims = []
  orden = []
  for i,q in enumerate(docs_consulta):
    ld = list(docs[i])
    if ld:
      s,o = fuerza_bruta(docs_base[ld], q, fs)
      sims.append(s)
      orden.append([ld[e] for e in o])
    else:
      sims.append([])
      orden.append([])

  return sims, orden

Buscamos documentos similares y examinamos un ejemplo de consulta y su correspondiente documento más similar.

In [17]:
tablas_ng_bin = [MinHashTable(2**18, 2, VOCMAX) for _ in range(n_tablas_ng)]
sims, orden = busca_pares_documentos(bolsas_base, bolsas_consultas, tablas_ng_bin, similitud_jaccard)
print("------ C O N S U L T A ------\n", consultas[1])
print("\n------ M Á S  S I M I L A R ------\n", base[list(orden[1])[0]])

ValueError: ignored

## Ejercicio
+ Evalúa la búsqueda con distintos valores de $r$ y $\eta$ usando la fórmula de 
$$
  l = \frac{log(0.5)}{log(1 - \eta^r)}
$$

+ Extiende la clase `MinHashTable` para que tome en cuenta las multiplicidades de las bolsas.